# Driver Aggression Neural Network (DANN)

Driver Aggression Neural Network is assigning an aggression value to a sorted set of sensory data. Driving is simulated in BeamNG v0.27 using their BeamNGpy open-source library.

In [ ]:
import pandas as pd

parquet_file_path = 'imu_data_2023_05_01_10_12_22.parquet'
data = pd.read_parquet(parquet_file_path)

## Modify training data

- Group together data recorded from the same sensor
- Take around 100-1000 recorded data without the aggression values
- Make aggression values the label of the dataset
- Create a lot of training data by chunking the sorted (by timestamp) records.

In [ ]:
import numpy as np

# Assuming 'data' is your Scheme
# data = pd.DataFrame(columns=[
#     'imuId',
#     'vehicleAggression',
#     'time',
#     'pos',
#     'dirX',
#     'dirY',
#     'dirZ',
#     'angVel',
#     'angAccel',
#     'mass',
#     'accRaw',
#     'accSmooth'
# ])

# Function to split the data into chunks
def split_into_chunks(data, chunk_size):
    return [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

# Group the data by 'imuId' and sort within each group by 'time'
grouped_data = data.groupby('imuId').apply(lambda x: x.sort_values('time')).reset_index(drop=True)

# Set the desired chunk size (number of records per chunk)
chunk_size = 200

# Split the data into chunks and assign the 'vehicleAggression' value as the label
training_data = []
for imu_id, group in grouped_data.groupby('imuId'):
    chunks = split_into_chunks(group, chunk_size)
    for chunk in chunks:
        if len(chunk) >= chunk_size:
            label = chunk['vehicleAggression'].iloc[0]
            first_timestamp = chunk['time'].iloc[0]
            adjusted_time = chunk['time'] - first_timestamp
            
            # Separate list columns into individual columns
            # pos_df = pd.DataFrame(chunk['pos'].tolist(), columns=['posX', 'posY', 'posZ'], index=chunk.index)
            # dir_x_df = pd.DataFrame(chunk['dirX'].tolist(), columns=['dirXX', 'dirXY', 'dirXZ'], index=chunk.index)
            # dir_y_df = pd.DataFrame(chunk['dirY'].tolist(), columns=['dirYX', 'dirYY', 'dirYZ'], index=chunk.index)
            # dir_z_df = pd.DataFrame(chunk['dirZ'].tolist(), columns=['dirZX', 'dirZY', 'dirZZ'], index=chunk.index)
            # acc_raw_df = pd.DataFrame(chunk['accRaw'].tolist(), columns=['accRawX', 'accRawY', 'accRawZ'], index=chunk.index)
            acc_smooth_df = pd.DataFrame(chunk['accSmooth'].tolist(), columns=['accSmoothX', 'accSmoothY', 'accSmoothZ'], index=chunk.index)
            # ang_vel_df = pd.DataFrame(chunk['angVel'].tolist(), columns=['angVelX', 'angVelY', 'angVelZ'], index=chunk.index)
            ang_accel_df = pd.DataFrame(chunk['angAccel'].tolist(), columns=['angAccelX', 'angAccelY', 'angAccelZ'], index=chunk.index)
            
            expanded_chunk = pd.concat(
                [
                    chunk,
                    # pos_df,
                    # dir_x_df,
                    # dir_y_df,
                    # dir_z_df,
                    # acc_raw_df,
                    acc_smooth_df,
                    # ang_vel_df,
                    ang_accel_df
                ],
                axis=1
            )
            
            updated_chunk = (
                expanded_chunk.assign(time=adjusted_time)
                .drop(['imuId', 'vehicleAggression', 'pos', 'dirX', 'dirY', 'dirZ', 'angVel', 'angAccel', 'accRaw', 'accSmooth'], axis=1)
            )
            
            training_data.append({'data': updated_chunk, 'label': label})

# Convert the list of dictionaries to a DataFrame
training_data_df = pd.DataFrame(training_data)

# Example of a single training set
# Set display options
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", 100)  # Set the maximum column width to 100 characters
pd.set_option("display.expand_frame_repr", False)
print(training_data_df.loc[0, 'data'])
pd.reset_option("all")


## Translating the training data to learn

In [ ]:
from sklearn.model_selection import train_test_split

# Get the data and labels from the training_data_df
X = np.stack(training_data_df['data'].apply(lambda x: x.to_numpy()).to_numpy())
y = training_data_df['label'].to_numpy()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

## Learn with PyTorch

- Create a TensorDataset
- Create a DataLoader, which shuffles the data
- Create a simple neural net (torch.nn.Sequential) which uses CUDA while training
- Train the neural net with the data provided
- Evaluate the net with the test data

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# # Pad sequences to the same length
# X_train_padded = pad_sequences(X_train, dtype='float32', padding='post')
# y_train_padded = pad_sequences(y_train, dtype='float32', padding='post')
# X_test_padded = pad_sequences(X_test, dtype='float32', padding='post')
# y_test_padded = pad_sequences(y_test, dtype='float32', padding='post')

# Create tensors from the padded data
X_train_tensor = torch.tensor(X_train).permute(0, 2, 1)
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test).permute(0, 2, 1)
y_test_tensor = torch.tensor(y_test)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
import torch.nn as nn

# Define the CNN architecture
class CNNRegressor(nn.Module):
    def __init__(self, input_channels, num_filters, kernel_size, pool_size, hidden_units, dropout_rate, device):
        super(CNNRegressor, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv1d(input_channels, num_filters, kernel_size),
            nn.BatchNorm1d(num_filters),
            nn.ReLU(),
            nn.MaxPool1d(pool_size)
        ).to(device)

        self.conv2 = nn.Sequential(
            nn.Conv1d(num_filters, num_filters * 2, kernel_size),
            nn.BatchNorm1d(num_filters * 2),
            nn.ReLU(),
            nn.MaxPool1d(pool_size)
        ).to(device)

        self.conv3 = nn.Sequential(
            nn.Conv1d(num_filters * 2, num_filters * 4, kernel_size),
            nn.BatchNorm1d(num_filters * 4),
            nn.ReLU(),
            nn.MaxPool1d(pool_size)
        ).to(device)

        self.flatten = nn.Flatten()

        conv1_out_size = (chunk_size - kernel_size + 1) // pool_size
        conv2_out_size = (conv1_out_size - kernel_size + 1) // pool_size
        conv3_out_size = (conv2_out_size - kernel_size + 1) // pool_size

        self.fc1 = nn.Sequential(
            nn.Linear(num_filters * 4 * conv3_out_size, hidden_units),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        ).to(device)

        self.fc2 = nn.Sequential(
            nn.Linear(hidden_units, hidden_units // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        ).to(device)

        self.fc3 = nn.Linear(hidden_units // 2, 1).to(device)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_channels = X_train.shape[2]

In [ ]:
import optuna
import random
import torch.optim as optim

def train_and_eval_net(trial):
    # Suggest hyperparameters using the trial object
    # num_filters = trial.suggest_int("num_filters", 128, 512)
    # kernel_size = trial.suggest_int("kernel_size", 3, 5)
    # pool_size = trial.suggest_int("pool_size", 2, 4)
    # hidden_units = trial.suggest_int("hidden_units", 32, 72)
    # dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.3)
    # learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    
    # GOAT (sometimes very good, sometimes bad)
    num_filters = 407
    kernel_size = 5
    pool_size = 3
    hidden_units = 49
    dropout_rate = 0.14489704854106614
    learning_rate = 0.0005565571451646218
    
    # Value: 0.008048494346439838
    # Params: {'num_filters': 435, 'kernel_size': 5, 'pool_size': 4, 'hidden_units': 61, 'dropout_rate': 0.1435561530697743, 'learning_rate': 0.0005436599406923652}
    # Consistently okay
    # num_filters = 435
    # kernel_size = 5
    # pool_size = 4
    # hidden_units = 61
    # dropout_rate = 0.1435561530697743
    # learning_rate = 0.0005436599406923652
    
    # Create the model
    net = CNNRegressor(input_channels, num_filters, kernel_size, pool_size, hidden_units, dropout_rate, device)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    num_epochs = 400
    
    net.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.float().to(device), labels.float().to(device)
    
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
        # print(f"Epoch {epoch+1}/{num_epochs} Loss: {running_loss/len(train_loader)}")

    net.eval()
    with torch.no_grad():
        total_difference = 0.0
        fake_difference = 0.0
        num_samples = 0
        test_loss = 0.0
        for inputs, labels in test_loader:
            inputs, labels = inputs.float().to(device), labels.float().to(device)
            outputs = net(inputs)

            # Calculate the absolute difference between the predicted and real labels
            difference = torch.abs(outputs.squeeze() - labels)

            # Update the total difference and the number of samples
            total_difference += difference.sum().item()
            num_samples += len(labels)

            loss = criterion(outputs.squeeze(), labels)
            test_loss += loss.item()

            # random outputs
            fake_outputs = torch.tensor([random.uniform(0.2, 0.6) for _ in range(len(outputs))], device=device)
            fake_diff = torch.abs(fake_outputs - labels)
            fake_difference += fake_diff.sum().item()

        print(f"Test Loss: {test_loss/len(test_loader)}")

        # Calculate the average absolute difference
        average_difference = total_difference / num_samples
        average_fake_difference = fake_difference / num_samples
        print(f"Average Absolute Difference: {average_difference}")
        # print(f"Average Absolute Fake Difference: {average_fake_difference}")
    
    return test_loss/len(test_loader)

train_and_eval_net(None)

## Hyperparameter optimization via Optuna

In [ ]:
# study = optuna.create_study(direction="minimize")
# study.optimize(train_and_eval_net, n_trials=100)  # You can adjust the number of trials depending on your computational resources

In [ ]:
# # Get the best 5 trials
# completed_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.COMPLETE])
# best_trials = sorted(completed_trials, key=lambda t: t.value)[:5]

# # Print the best 5 trials' parameters and their respective values
# for i, trial in enumerate(best_trials):
#     print(f"Best trial {i + 1}:")
#     print(f"  Value: {trial.value}")
#     print(f"  Params: {trial.params}")